# Hurricane Ike -- Using Python Tools

This notebook is a modified copy of a different "official" GeoClaw example. THE FOLLOWING INFORMATION IS NOT ACCURATE!

This [Jupyter notebook](http://www.jupyter.org) can be found in [collection of Clawpack apps](http://www.clawpack.org/apps.html) as the file [`$CLAW/apps/notebooks/geoclaw/chile2010a/chile2010a.ipynb`](https://github.com/clawpack/apps/tree/master/notebooks/geoclaw/chile2010a/chile2010a.ipynb).  
To run this notebook, [install Clawpack](http://www.clawpack.org/installing.html), and clone the [apps repository](https://github.com/clawpack/apps).
A static view of this and other notebooks can be found in the [Clawpack Gallery of Jupyter notebooks](http://www.clawpack.org/gallery/notebooks.html).

This notebook walks through several experiments you can do in the directory `$CLAW/apps/notebooks/geoclaw/chile2010a`.

The experiments are meant to be done by modifying the file `setrun.py` using an editor and saving the file, and then giving the command

    make .plots

in a shell terminal.  This will re-run the code with the modified parameters and produce a new set of plots that can be viewed with a web browser.  

This notebook explains a set of experiments you might do and also shows some of the resulting plots.  In order to produce these plots, the GeoClaw code has been run from the notebook, so below you can also see how to work in this mode.  

### Version

Animation revised 2020-04-09 to run with v5.7.0

## Notebook setup

The next few cells are need to set things up for running code in the notebook.  Skip to **Experiment 1** to get started with the experiments.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from clawpack.clawutil import nbtools
from clawpack.visclaw import animation_tools
from IPython.display import HTML

### Choose how to display animations:

Using `anim.to_jshtml()` gives animations similar to what you see in the html files if you do `make plots`, but you may prefer the `anim.to_html5_video()` option.  See the [matplotlib.animation.Animation documentation](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.animation.Animation.html) for more information, also on how to save an animation as a separate file.

In [3]:
def show_anim(anim):
    html_version = HTML(anim.to_jshtml())
    #html_version = HTML(anim.to_html5_video())
    return html_version

## Compile and run the GeoClaw code

In [4]:
nbtools.make_exe(new=True, verbose=False)  # compile xgeoclaw

In [38]:
import numpy as np
def setgeo(rundata):
    """
    Set GeoClaw specific runtime parameters.
    For documentation see ....
    """

    geo_data = rundata.geo_data

    # == Physics ==
    geo_data.gravity = 9.81
    geo_data.coordinate_system = 2
    geo_data.earth_radius = 6367.5e3
    geo_data.rho = 1025.0
    geo_data.rho_air = 1.15
    geo_data.ambient_pressure = 101.3e3

    # == Forcing Options
    geo_data.coriolis_forcing = True
    geo_data.friction_forcing = True
    geo_data.friction_depth = 1e10

    # == Algorithm and Initial Conditions ==
    # Note that in the original paper due to gulf summer swelling this was set
    # to 0.28
    geo_data.sea_level = 0.0
    geo_data.dry_tolerance = 1.e-2

    # Refinement Criteria
    refine_data = rundata.refinement_data
    refine_data.wave_tolerance = 1.0
    refine_data.speed_tolerance = [1.0, 2.0, 3.0, 4.0]
    refine_data.variable_dt_refinement_ratios = True

    # == settopo.data values ==
    topo_data = rundata.topo_data
    topo_data.topofiles = []
    # for topography, append lines of the form
    #   [topotype, fname]
    # See regions for control over these regions, need better bathy data for
    # the smaller domains
    clawutil.data.get_remote_file(
           "http://www.columbia.edu/~ktm2132/bathy/gulf_caribbean.tt3.tar.bz2")
    topo_path = os.path.join(scratch_dir, 'gulf_caribbean.tt3')
    topo_data.topofiles.append([3, topo_path])

    # == setfixedgrids.data values ==
    rundata.fixed_grid_data.fixedgrids = []
    # for fixed grids append lines of the form
    # [t1,t2,noutput,x1,x2,y1,y2,xpoints,ypoints,\
    #  ioutarrivaltimes,ioutsurfacemax]

    # ================
    #  Set Surge Data
    # ================
    data = rundata.surge_data

    # Source term controls
    data.wind_forcing = True
    data.drag_law = 1
    data.pressure_forcing = True

    data.display_landfall_time = True

    # AMR parameters, m/s and m respectively
    data.wind_refine = [20.0, 40.0, 60.0]
    data.R_refine = [60.0e3, 40e3, 20e3]

    # Storm parameters - Parameterized storm (Holland 1980)
    data.storm_specification_type = 'holland80'  # (type 1)
    data.storm_file = os.path.expandvars(os.path.join(os.getcwd(),
                                         'ike.storm'))

    # Convert ATCF data to GeoClaw format
    clawutil.data.get_remote_file(
                   "http://ftp.nhc.noaa.gov/atcf/archive/2008/bal092008.dat.gz")
    atcf_path = os.path.join(scratch_dir, "bal092008.dat")
    # Note that the get_remote_file function does not support gzip files which
    # are not also tar files.  The following code handles this
    with gzip.open(".".join((atcf_path, 'gz')), 'rb') as atcf_file,    \
            open(atcf_path, 'w') as atcf_unzipped_file:
        atcf_unzipped_file.write(atcf_file.read().decode('ascii'))

    # Uncomment/comment out to use the old version of the Ike storm file
    # ike = Storm(path="old_ike.storm", file_format="ATCF")
    ike = Storm(path=atcf_path, file_format="ATCF")

    # Calculate landfall time - Need to specify as the file above does not
    # include this info (9/13/2008 ~ 7 UTC)
    ike.time_offset = datetime.datetime(2008, 9, 13, 7)
    
#     trajectory = ike.eye_location
#     traj_x = trajectory[:,0]
#     traj_y = trajectory[:,1]
#     new_traj_x = np.linspace(traj_x[0],traj_x[-1],len(traj_x))
#     new_traj_y = np.linspace(traj_y[0],traj_y[-1],len(traj_y))
#     new_traj_x = traj_x+12
#     new_traj_y = traj_y
#     new_trajectory=trajectory*0
#     new_trajectory[:,0]=new_traj_x
#     new_trajectory[:,1]=new_traj_y
#     new_trajectory = np.around(new_trajectory,1)
#     ike.eye_location = new_trajectory

    ike.write(data.storm_file, file_format='geoclaw')

    # =======================
    #  Set Variable Friction
    # =======================
    data = rundata.friction_data

    # Variable friction
    data.variable_friction = True

    # Region based friction
    # Entire domain
    data.friction_regions.append([rundata.clawdata.lower,
                                  rundata.clawdata.upper,
                                  [np.infty, 0.0, -np.infty],
                                  [0.030, 0.022]])

    # La-Tex Shelf
    data.friction_regions.append([(-98, 25.25), (-90, 30),
                                  [np.infty, -10.0, -200.0, -np.infty],
                                  [0.030, 0.012, 0.022]])

    return rundata
    # end of function setgeo
    # ----------------------

In [39]:
import os
import datetime
import shutil
import gzip

import numpy as np

from clawpack.geoclaw.surge.storm import Storm
import clawpack.clawutil as clawutil


# Time Conversions
def days2seconds(days):
    return days * 60.0**2 * 24.0


# Scratch directory for storing topo and storm files:
scratch_dir = os.path.join(os.environ["CLAW"], 'geoclaw', 'scratch')


# ------------------------------
def setrun(claw_pkg='geoclaw'):

    """
    Define the parameters used for running Clawpack.

    INPUT:
        claw_pkg expected to be "geoclaw" for this setrun.

    OUTPUT:
        rundata - object of class ClawRunData

    """

    from clawpack.clawutil import data

    assert claw_pkg.lower() == 'geoclaw',  "Expected claw_pkg = 'geoclaw'"

    num_dim = 2
    rundata = data.ClawRunData(claw_pkg, num_dim)

    # ------------------------------------------------------------------
    # Standard Clawpack parameters to be written to claw.data:
    #   (or to amr2ez.data for AMR)
    # ------------------------------------------------------------------
    clawdata = rundata.clawdata  # initialized when rundata instantiated

    # Set single grid parameters first.
    # See below for AMR parameters.

    # ---------------
    # Spatial domain:
    # ---------------

    # Number of space dimensions:
    clawdata.num_dim = num_dim

    # Lower and upper edge of computational domain:
    clawdata.lower[0] = -99.0      # west longitude
    clawdata.upper[0] = -70.0      # east longitude

    clawdata.lower[1] = 8.0       # south latitude
    clawdata.upper[1] = 32.0      # north latitude

    # Number of grid cells:
    degree_factor = 4  # (0.25º,0.25º) ~ (25237.5 m, 27693.2 m) resolution
    clawdata.num_cells[0] = int(clawdata.upper[0] - clawdata.lower[0]) \
        * degree_factor
    clawdata.num_cells[1] = int(clawdata.upper[1] - clawdata.lower[1]) \
        * degree_factor

    # ---------------
    # Size of system:
    # ---------------

    # Number of equations in the system:
    clawdata.num_eqn = 3

    # Number of auxiliary variables in the aux array (initialized in setaux)
    # First three are from shallow GeoClaw, fourth is friction and last 3 are
    # storm fields
    clawdata.num_aux = 3 + 1 + 3

    # Index of aux array corresponding to capacity function, if there is one:
    clawdata.capa_index = 2

    # -------------
    # Initial time:
    # -------------
    clawdata.t0 = -days2seconds(3)

    # Restart from checkpoint file of a previous run?
    # If restarting, t0 above should be from original run, and the
    # restart_file 'fort.chkNNNNN' specified below should be in
    # the OUTDIR indicated in Makefile.

    clawdata.restart = False               # True to restart from prior results
    clawdata.restart_file = 'fort.chk00006'  # File to use for restart data

    # -------------
    # Output times:
    # --------------

    # Specify at what times the results should be written to fort.q files.
    # Note that the time integration stops after the final output time.
    # The solution at initial time t0 is always written in addition.

    clawdata.output_style = 1

    if clawdata.output_style == 1:
        # Output nout frames at equally spaced times up to tfinal:
        clawdata.tfinal = days2seconds(1)
        recurrence = 4
        clawdata.num_output_times = int((clawdata.tfinal - clawdata.t0) *
                                        recurrence / (60**2 * 24))

        clawdata.output_t0 = True  # output at initial (or restart) time?

    elif clawdata.output_style == 2:
        # Specify a list of output times.
        clawdata.output_times = [0.5, 1.0]

    elif clawdata.output_style == 3:
        # Output every iout timesteps with a total of ntot time steps:
        clawdata.output_step_interval = 1
        clawdata.total_steps = 1
        clawdata.output_t0 = True

    clawdata.output_format = 'ascii'      # 'ascii' or 'binary'
    clawdata.output_q_components = 'all'   # could be list such as [True,True]
    clawdata.output_aux_components = 'all'
    clawdata.output_aux_onlyonce = False    # output aux arrays only at t0

    # ---------------------------------------------------
    # Verbosity of messages to screen during integration:
    # ---------------------------------------------------

    # The current t, dt, and cfl will be printed every time step
    # at AMR levels <= verbosity.  Set verbosity = 0 for no printing.
    #   (E.g. verbosity == 2 means print only on levels 1 and 2.)
    clawdata.verbosity = 0

    # --------------
    # Time stepping:
    # --------------

    # if dt_variable==1: variable time steps used based on cfl_desired,
    # if dt_variable==0: fixed time steps dt = dt_initial will always be used.
    clawdata.dt_variable = True

    # Initial time step for variable dt.
    # If dt_variable==0 then dt=dt_initial for all steps:
    clawdata.dt_initial = 0.016

    # Max time step to be allowed if variable dt used:
    clawdata.dt_max = 1e+99

    # Desired Courant number if variable dt used, and max to allow without
    # retaking step with a smaller dt:
    clawdata.cfl_desired = 0.75
    clawdata.cfl_max = 1.0

    # Maximum number of time steps to allow between output times:
    clawdata.steps_max = 5000

    # ------------------
    # Method to be used:
    # ------------------

    # Order of accuracy:  1 => Godunov,  2 => Lax-Wendroff plus limiters
    clawdata.order = 2

    # Use dimensional splitting? (not yet available for AMR)
    clawdata.dimensional_split = 'unsplit'

    # For unsplit method, transverse_waves can be
    #  0 or 'none'      ==> donor cell (only normal solver used)
    #  1 or 'increment' ==> corner transport of waves
    #  2 or 'all'       ==> corner transport of 2nd order corrections too
    clawdata.transverse_waves = 2

    # Number of waves in the Riemann solution:
    clawdata.num_waves = 3

    # List of limiters to use for each wave family:
    # Required:  len(limiter) == num_waves
    # Some options:
    #   0 or 'none'     ==> no limiter (Lax-Wendroff)
    #   1 or 'minmod'   ==> minmod
    #   2 or 'superbee' ==> superbee
    #   3 or 'mc'       ==> MC limiter
    #   4 or 'vanleer'  ==> van Leer
    clawdata.limiter = ['mc', 'mc', 'mc']

    clawdata.use_fwaves = True    # True ==> use f-wave version of algorithms

    # Source terms splitting:
    #   src_split == 0 or 'none'
    #      ==> no source term (src routine never called)
    #   src_split == 1 or 'godunov'
    #      ==> Godunov (1st order) splitting used,
    #   src_split == 2 or 'strang'
    #      ==> Strang (2nd order) splitting used,  not recommended.
    clawdata.source_split = 'godunov'

    # --------------------
    # Boundary conditions:
    # --------------------

    # Number of ghost cells (usually 2)
    clawdata.num_ghost = 2

    # Choice of BCs at xlower and xupper:
    #   0 => user specified (must modify bcN.f to use this option)
    #   1 => extrapolation (non-reflecting outflow)
    #   2 => periodic (must specify this at both boundaries)
    #   3 => solid wall for systems where q(2) is normal velocity

    clawdata.bc_lower[0] = 'extrap'
    clawdata.bc_upper[0] = 'extrap'

    clawdata.bc_lower[1] = 'extrap'
    clawdata.bc_upper[1] = 'extrap'

    # Specify when checkpoint files should be created that can be
    # used to restart a computation.

    clawdata.checkpt_style = 0

    if clawdata.checkpt_style == 0:
        # Do not checkpoint at all
        pass

    elif np.abs(clawdata.checkpt_style) == 1:
        # Checkpoint only at tfinal.
        pass

    elif np.abs(clawdata.checkpt_style) == 2:
        # Specify a list of checkpoint times.
        clawdata.checkpt_times = [0.1, 0.15]

    elif np.abs(clawdata.checkpt_style) == 3:
        # Checkpoint every checkpt_interval timesteps (on Level 1)
        # and at the final time.
        clawdata.checkpt_interval = 5

    # ---------------
    # AMR parameters:
    # ---------------
    amrdata = rundata.amrdata

    # max number of refinement levels:
    amrdata.amr_levels_max = 2

    # List of refinement ratios at each level (length at least mxnest-1)
    amrdata.refinement_ratios_x = [2, 2, 2, 6, 16]
    amrdata.refinement_ratios_y = [2, 2, 2, 6, 16]
    amrdata.refinement_ratios_t = [2, 2, 2, 6, 16]

    # Specify type of each aux variable in amrdata.auxtype.
    # This must be a list of length maux, each element of which is one of:
    #   'center',  'capacity', 'xleft', or 'yleft'  (see documentation).

    amrdata.aux_type = ['center', 'capacity', 'yleft', 'center', 'center',
                        'center', 'center']

    # Flag using refinement routine flag2refine rather than richardson error
    amrdata.flag_richardson = False    # use Richardson?
    amrdata.flag2refine = True

    # steps to take on each level L between regriddings of level L+1:
    amrdata.regrid_interval = 3

    # width of buffer zone around flagged points:
    # (typically the same as regrid_interval so waves don't escape):
    amrdata.regrid_buffer_width = 2

    # clustering alg. cutoff for (# flagged pts) / (total # of cells refined)
    # (closer to 1.0 => more small grids may be needed to cover flagged cells)
    amrdata.clustering_cutoff = 0.700000

    # print info about each regridding up to this level:
    amrdata.verbosity_regrid = 0

    #  ----- For developers -----
    # Toggle debugging print statements:
    amrdata.dprint = False      # print domain flags
    amrdata.eprint = False      # print err est flags
    amrdata.edebug = False      # even more err est flags
    amrdata.gprint = False      # grid bisection/clustering
    amrdata.nprint = False      # proper nesting output
    amrdata.pprint = False      # proj. of tagged points
    amrdata.rprint = False      # print regridding summary
    amrdata.sprint = False      # space/memory output
    amrdata.tprint = False      # time step reporting each level
    amrdata.uprint = False      # update/upbnd reporting

    # More AMR parameters can be set -- see the defaults in pyclaw/data.py

    # == setregions.data values ==
    regions = rundata.regiondata.regions
    # to specify regions of refinement append lines of the form
    #  [minlevel,maxlevel,t1,t2,x1,x2,y1,y2]
    # Gauges from Ike AWR paper (2011 Dawson et al)
    rundata.gaugedata.gauges.append([1, -95.04, 29.07,
                                     rundata.clawdata.t0,
                                     rundata.clawdata.tfinal])
    rundata.gaugedata.gauges.append([2, -94.71, 29.28,
                                     rundata.clawdata.t0,
                                     rundata.clawdata.tfinal])
    rundata.gaugedata.gauges.append([3, -94.39, 29.49,
                                     rundata.clawdata.t0,
                                     rundata.clawdata.tfinal])
    rundata.gaugedata.gauges.append([4, -94.13, 29.58,
                                     rundata.clawdata.t0,
                                     rundata.clawdata.tfinal])

    # Force the gauges to also record the wind and pressure fields
    rundata.gaugedata.aux_out_fields = [4, 5, 6]

    # ------------------------------------------------------------------
    # GeoClaw specific parameters:
    # ------------------------------------------------------------------
    rundata = setgeo(rundata)

    return rundata
    # end of function setrun
    # ----------------------

In [40]:
# # modify setrun parameters
# from setrun import setrun

In [41]:
# create *.data files from parameters in setrun.py

rundata = setrun()
rundata.write()


In [42]:
# from clawpack.geoclaw.surge.storm import Storm
# ike = Storm("ike.storm", file_format='GeoClaw')

In [43]:
# trajectory = ike.eye_location
# traj_x = trajectory[0,:]
# traj_y = trajectory[1,:]
# new_traj_x = np.linspace(traj_x[0],traj_x[-1],len(traj_x))
# new_traj_y = np.linspace(traj_y[0],traj_y[-1],len(traj_y))
# new_trajectory=trajectory*0
# new_trajectory[0,:]=new_traj_x
# new_trajectory[1,:]=new_traj_y
# ike.eye_location = new_trajectory
# # print(trajectory.shape,new_trajectory.shape)
# # print(ike.eye_location.shape)
# ike.write('ike2.storm',file_format = 'geoclaw')

In [44]:
# data = rundata.surge_data

# # Source term controls
# data.wind_forcing = True
# data.drag_law = 1
# data.pressure_forcing = True

# data.display_landfall_time = True

# # AMR parameters, m/s and m respectively
# data.wind_refine = [20.0, 40.0, 60.0]
# data.R_refine = [60.0e3, 40e3, 20e3]

# # Storm parameters - Parameterized storm (Holland 1980)
# data.storm_specification_type = 'holland80'  # (type 1)
# data.storm_file = os.path.expandvars(os.path.join(os.getcwd(),
#                                      'ike.storm'))

# # Convert ATCF data to GeoClaw format
# clawutil.data.get_remote_file(
#                "http://ftp.nhc.noaa.gov/atcf/archive/2008/bal092008.dat.gz")
# atcf_path = os.path.join(scratch_dir, "bal092008.dat")
# # Note that the get_remote_file function does not support gzip files which
# # are not also tar files.  The following code handles this
# with gzip.open(".".join((atcf_path, 'gz')), 'rb') as atcf_file,    \
#         open(atcf_path, 'w') as atcf_unzipped_file:
#     atcf_unzipped_file.write(atcf_file.read().decode('ascii'))

# # Uncomment/comment out to use the old version of the Ike storm file
# # ike = Storm(path="old_ike.storm", file_format="ATCF")
# ike = Storm(path=atcf_path, file_format="ATCF")

# # Calculate landfall time - Need to specify as the file above does not
# # include this info (9/13/2008 ~ 7 UTC)
# ike.time_offset = datetime.datetime(2008, 9, 13, 7)

# trajectory = ike.eye_location
# traj_x = trajectory[0,:]
# traj_y = trajectory[1,:]
# new_traj_x = np.linspace(traj_x[0],traj_x[-1],len(traj_x))
# new_traj_y = np.linspace(traj_y[0],traj_y[-1],len(traj_y))
# new_trajectory=trajectory*0
# new_trajectory[0,:]=new_traj_x
# new_trajectory[1,:]=new_traj_y
# ike.eye_location = new_trajectory

# ike.write(data.storm_file, file_format='geoclaw')

In [45]:
## This is not needed for ike
#run maketopo.py  # download the topo file and create the dtopo file

In [46]:
# Run the code with the original parameter settings
outdir,plotdir = nbtools.make_output_and_plots(verbose=True)

Executing shell command:   make output OUTDIR=_output
Done...  Check this file to see output:


/home/jovyan/data/hydroinformatics/ike/run_output.txt

Executing shell command:   make plots OUTDIR=_output PLOTDIR=_plots
Done...  Check this file to see output:


/home/jovyan/data/hydroinformatics/ike/plot_output.txt

View plots created at this link:


/home/jovyan/data/hydroinformatics/ike/_plots/_PlotIndex.html

## Experiment 1 -- One-level run on a coarse grid

As a first test, compile and run the code using the parameter values in the original `setrun.py` file in this directory.  (If you have been experimenting with it and want to recover the original, this same file is also in `setrun_original.py`.)

First download a topography file and create the seafloor deformation file if necessary.  **In a terminal window:**

    make topo
    
Compile the code:

    make .exe
    
Run the code and create plots:

    make .plots
    
Then you should be able to open the file `_plots/_PlotIndex.html` in a web browser and see the results, including an animation that looks like this:

In [47]:
anim = animation_tools.animate_from_plotdir(plotdir,figno=1001);

In [48]:
show_anim(anim)

2021-07-09 13:50:15,199 INFO CLAW: Animation.save using <class 'matplotlib.animation.HTMLWriter'>
2021-07-09 13:50:15,201 INFO CLAW: figure size in inches has been adjusted from 6.0 x 4.837988826815643 to 6.0 x 4.833333333333333


## Experiment 2 - Add Level 2 grids with adaptive refinement

Experiment 1 above was run without grid refinement on a very coarse 30 by 30 grid -- each grid cell is 2 degrees (approximately 220 km) on a side and so the result is not very useful.

You could rerun the code with a finer grid by changing the following lines in `setrun.py`:

    # Number of grid cells: Coarsest grid
    clawdata.num_cells[0] = 30
    clawdata.num_cells[1] = 30
    
But over much of the domain nothing is happening, so a more efficient approach is to leave the resolution of this coarsest level 1 grid alone and instead add an additional level of refinement only where the wave is present.

Modify the lines (starting at line 282)

    # max number of refinement levels:
    amrdata.amr_levels_max = 1

to increase the maximum level allowed from 1 to 2:

    # max number of refinement levels:
    amrdata.amr_levels_max = 2
    
Note that the next lines read:

    # List of refinement ratios at each level:
    amrdata.refinement_ratios_x = [2]
    amrdata.refinement_ratios_y = [2]
    amrdata.refinement_ratios_t = [2]

This means that Level 2 grids will be 2 times finer than Level 1 grids in each direction. They will also be 2 times finer in t, meaning two time steps must be taken on each Level 2 grid for every time step on Level 1.  This is handled automatically within GeoClaw.

Now save `setrun.py` and re-execute `make .plots` to recreate the `_plots` directory.

The results should look like what is shown below, after making the same change in the notebook version of the code.

In [ ]:
rundata.amrdata.amr_levels_max = 1
rundata.write()
outdir,plotdir = nbtools.make_output_and_plots(verbose=True)

In [ ]:
anim = animation_tools.animate_from_plotdir(plotdir,figno = 1001)
show_anim(anim)

Notice several things in this animation:

 - There are generally several patches of grids at Level 2 that might or might not be contiguous
 - The location of the patches changes with time to follow (parts of) the tsunami.
 - Some parts of the tsunami are not resolved on Level 2 at later times.
 
## Experiment 3: Changing the refinement criterion
 
There are several parameters in `setrun.py` that control the behavior of the AMR algorithms.  For example, the movement of the patches is due to the fact that re-gridding is performed every few time steps and the number of steps between regridding can be adjusted.  

When re-gridding is performed, some criteria are used to determine what regions need to be refined.  In this example we are simply flagging coarse cells as needing refinement whereever the amplitude of the surface (relative to sea level) is above some tolerance.  By making the tolerance smaller, we will cause more of the domain to be refined to Level 2 at letter times. 

Find the line 

    refinement_data.wave_tolerance = 0.1

in `setrun.py` and change it to:

    refinement_data.wave_tolerance = 0.02
    
Now re-run the code and you should see that it refines much more of the wave.  In fact at later times it refines almost the entire domain.

Below we make the same change in the notebook to display the new results.

In [ ]:
rundata.refinement_data.wave_tolerance = 0.02
rundata.write()
outdir,plotdir = nbtools.make_output_and_plots(verbose=False)
anim = animation_tools.animate_from_plotdir(plotdir)
show_anim(anim)

## Experiment 4 -- Adding a third level

Let's add a third level of AMR, refining by another factor of two in each dimension going from Level 2 to Level 3.  Do this by fixing `setrun.py` to have these lines:

    # max number of refinement levels:
    amrdata.amr_levels_max = 3

    # List of refinement ratios at each level:
    amrdata.refinement_ratios_x = [2,2]
    amrdata.refinement_ratios_y = [2,2]
    amrdata.refinement_ratios_t = [2,2]
    
Note that you have to add another component to the `refinement_ratios` to give the refinement factor from Level 2 to Level 3.  The refinement ratios do not have to be 2, they can be any integer.  In general you should refine by the same factors in `x` and `y`.

(*Note:* The refinement factors in `t` are actually ignored because of another line in `setrun.py` that tell GeoClaw to choose time steps appropriately at each level.)

If you make this change and run the code again, you should plots like those shown below.

Note that at Level 3 we are not plotting the grid lines (which would be so dense they hide the wave). Instead only the patch boundaries are shown.  Plotting behavior is controlled by parameters set in `setplot.py` explored later.

In [ ]:
rundata.amrdata.amr_levels_max = 3
rundata.amrdata.refinement_ratios_x = [2,2]
rundata.amrdata.refinement_ratios_y = [2,2]
rundata.amrdata.refinement_ratios_t = [2,2]
rundata.write()
outdir,plotdir = nbtools.make_output_and_plots(verbose=False)
anim = animation_tools.animate_from_plotdir(plotdir)
show_anim(anim)

## Experiment 5 -- Restricting or forcing refinement in "regions"

In the last experiment we have resolved the tsunami fairly well everywhere.  The refinement criterion is simply the amplitude of the wave. 

In many applications we do not need to refine equally well everywhere, e.g. if we are only interested in modeling the effect of the tsunami on one particular coastline.  We would then like to restrict the regions where refinement to a certain level is allowed.  

We might also have particular regions where we want to **force** refinement to some level, over some time period.  For example around the earthquake source region, or a region on the coast where we want fine grids even if the tsunami is small there.

GeoClaw allows specifying rectangular regions in space-time over which refinement is required to be at least to Level $L_1$ and at most to level $L_2$.  Multiple regions can be specified, and this can be used in conjunction with flagging based on amplitude (whether a point is refined only to $L_1$ or to some higher level $\leq L_2$ depends on the amplitude).  If a point lies in more than one region, the maximum of $L_1$ values and the maximum of the $L_2$ values for each region are used as limits.  See [the documentation](http://www.clawpack.org/refinement.html#specifying-amr-regions) for more information.

In this experiment we will introduce 3 regions in this example.  Find these lines in `setrun.py`:

    rundata.regiondata.regions = []
    # to specify regions of refinement append lines of the form
    #  [minlevel,maxlevel,t1,t2,x1,x2,y1,y2]

    if 0:
        # Allow only level 1 as default everywhere:
        rundata.regiondata.regions.append([1, 1, 0., 1e9, -180, 180, -90, 90])

        # Force refinement around earthquake source region for first hour:
        rundata.regiondata.regions.append([3, 3, 0., 3600., -85,-72,-38,-25])

        # Allow up to level 3 in northeastern part of domain:
        rundata.regiondata.regions.append([1, 3, 0., 1.e9, -90,-60,-30,0])
        
These lines are effectively commented-out by the `if 0:`, so simply change `0` to `1` to specify the 3 regions. (`0=False, 1=True` in this context)

 - The first region simply sets the default to a maximum of 1 level anywhere.  
 - The second region forces refinement to Level 3 in the region around the earthquake source, but only for the first 3600 seconds.  
 - The third region allows up to 3 levels over the northeastern part of the domain ($x>-90,~y>-30$) which might be appropriate if we were only interested in the impact on the Peru coast, for example.
 
We can set these same regions in the notebook in order to produce the plots you should observe when you rerun the code:

In [ ]:
rundata.regiondata.regions = []  # empty list of regions

# Allow only level 1 as default everywhere:
rundata.regiondata.regions.append([1, 1, 0., 1e9, -180, 180, -90, 90])

# Force refinement around earthquake source region for first hour:
rundata.regiondata.regions.append([3, 3, 0., 3600., -85,-72,-38,-25])

# Allow up to level 3 in northeastern part of domain:
rundata.regiondata.regions.append([1, 3, 0., 1.e9, -90,-60,-30,0])

rundata.write()
outdir,plotdir = nbtools.make_output_and_plots(verbose=False)
anim = animation_tools.animate_from_plotdir(plotdir)
show_anim(anim)

## Next step:

The directory `$CLAW/apps/notebooks/geoclaw/chile2010b` explores this example further by adding some gauges to capture time series of the solution.